In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [1]:
import pandas as pd

df = pd.read_feather("/content/drive/MyDrive/home_assignment.feather").sample(frac=0.3)

## **Data Preparation**

Let's look at the target variable

In [2]:
# Count the number of values greater than x
count_greater_than_x = (df['TLJYWBE'] > 1e-5).sum()

# Count the number of values less than x
count_less_than_x = (df['TLJYWBE'] < 1e-5).sum()

print(f"Count of values greater than {1e-5}: {count_greater_than_x}")
print(f"Count of values less than {1e-5}: {count_less_than_x}")

Count of values greater than 1e-05: 64
Count of values less than 1e-05: 709454


Create a data report to see the distributions, dtypes and more

In [5]:
from ydata_profiling import ProfileReport

# Create a ProfileReport
profile = ProfileReport(df_filtered.sample(frac = 0.1), title="Pandas Profiling Report", minimal=True)

# Save the report to an HTML file
profile.to_file("EDA.html")

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Try to reduce number of columns - first let's remove columns where we think the model won't learn to much from (e.g where most data is null or constant value)

In [ ]:
# Identify columns where NaN/nulls makes up more than 70% of the data
null_dominated_columns = [col for col in df.columns if df[col].isna().mean() > 0.70]
nan_dominated_columns = [col for col in df.columns if df[col].isnull().mean() > 0.70]

# Identify columns where 0 makes up more than 70% of the data
zero_dominated_columns = [col for col in df.columns if (df[col] == 0).mean() > 0.70]

# Identify columns with 1 dominate value
value_dominated_columns = [col for col in df.columns if df[col].value_counts(normalize=True).max() > 0.9]

columns_to_drop = list(set(value_dominated_columns + zero_dominated_columns + nan_dominated_columns))

# Drop these columns from the DataFrame
df_filtered = df.drop(columns=columns_to_drop)

# Display the columns that were dropped
print(f"Columns with constant values or dominated by zeros: {columns_to_drop}")
print(f"Original shape: {df.shape}")
print(f"Reduced shape: {df_filtered.shape}")

I want to check the columns I removed and if they are corr witht he target variable - is so, I will reinstate those who are highly correleted

In [7]:
dropped_columns_df = df[columns_to_drop]

# Calculate the correlation with the outcome
correlations = dropped_columns_df.corrwith(df['TLJYWBE'])

# Display the correlation values
print(correlations)

# Optionally, filter and display only correlations above a certain threshold (e.g., |0.1|)
significant_correlations = correlations[abs(correlations) > 0.1]
print("Columns with significant correlation with the outcome:")
print(significant_correlations)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


AXVFXOV         NaN
FIJMBVX         NaN
UZXVNSW   -0.000016
TNLTITB         NaN
VUTOMKG   -0.021770
OELGKKQ         NaN
ZAIRSDG   -0.000959
HFHITLP    0.001113
HTTHEYH         NaN
FSBLFRV         NaN
GGNNDQY         NaN
QXTPTQN         NaN
ILHBFRB   -0.000016
OOBREVT   -0.000335
WWHSAAX         NaN
VWPFOKV         NaN
HNDGNRG    0.000772
LGRGANO         NaN
BQLHRUO         NaN
MSVWNFK    0.021179
BADESAP   -0.000539
QTIPFME         NaN
ZSCIZXU    0.001083
FFRECAD   -0.000371
XKLWARB    0.011702
BZRZXSR         NaN
HGWWOZY         NaN
EBEXNTO         NaN
ZMOCACF   -0.000405
BTDJSVV    0.045649
PCIXTYP         NaN
SHEGNUT   -0.000093
LVBGDZB         NaN
LYCIQTI         NaN
SWMNPFK         NaN
ILGYBTA         NaN
OLOIPWS         NaN
KRMCXMU         NaN
VMYGDBS         NaN
VRXSNDL         NaN
TQZKVZQ   -0.000472
AUNDDNZ    0.000674
FIGZKCY         NaN
NMCAKWK    0.001861
XZSIRUV         NaN
AHMUWMD         NaN
BKAHOMJ   -0.000324
HVHDNTE   -0.000016
VXQLENI         NaN
ZBSCEOT         NaN


I found that  'LNZGNNA', 'YTYOTNN' and 'KWGZVQN' are corr with the target variable so I decide to include them back

In [3]:
# I exclude columns that are correlated with the target variabel, even if thay are unbalanced
# Since they are highly corr, the model might pick on the relashionship to make better predictions
exclude_columns = ['LNZGNNA', 'YTYOTNN', 'KWGZVQN']

all_columns_to_drop = list(set(value_dominated_columns + zero_dominated_columns + nan_dominated_columns))

# Remove the columns that should be excluded
columns_to_drop = [col for col in all_columns_to_drop if col not in exclude_columns]

# Drop these columns from the DataFrame
df_filtered = df.drop(columns=columns_to_drop)

# Display the columns that were dropped
print(f"Columns with constant values or dominated by zeros: {columns_to_drop}")
print(f"Original shape: {df.shape}")
print(f"Reduced shape: {df_filtered.shape}")

Columns with constant values or dominated by zeros: ['JWJHZJJ', 'PGENXWU', 'CDSPXSX', 'BADTSBX', 'SWMNPFK', 'CUULBWE', 'UANVOYU', 'OPYOPFC', 'OETLBRB', 'JZVNCXT', 'NMCAKWK', 'AJOFIZJ', 'BZRZXSR', 'HVHDNTE', 'ZBSCEOT', 'CNECOOI', 'BXBUVRU', 'IPLONQP', 'FZHCPXN', 'VMYGDBS', 'RHCAZIU', 'VUBFVUN', 'VHLPUWU', 'LWFSOBB', 'ZMOCACF', 'FGRQLRJ', 'ILHBFRB', 'NMWUWJL', 'UZXVNSW', 'BKAHOMJ', 'TDMCRGU', 'QNIBUZZ', 'ABKUZXQ', 'MKVXAUF', 'HTMOHRO', 'FIGZKCY', 'BBMREOD', 'RQTEIMY', 'VXQLENI', 'XZJYNXP', 'TXEIVQY', 'KXGNEZR', 'ZQMONHC', 'TQSLCKT', 'FRPZZAO', 'RNCEKOH', 'EJLLSVL', 'AQYGYOE', 'GJFZDWF', 'VWPFOKV', 'CPZMPBY', 'WPJXHBR', 'WTARSLQ', 'KKKSVYN', 'MOYGMAB', 'YLYAAGH', 'FIJMBVX', 'RGDZIYI', 'BFYTKKS', 'WCYWJXK', 'OAEOBHM', 'CYGFXWC', 'OBFPKIQ', 'XUCVAEV', 'QRYQRQW', 'IJEXXXL', 'IJOSAYZ', 'FMWCHDN', 'QXTPTQN', 'ADOIOYN', 'SIJOOGR', 'FBTIPCA', 'GGNNDQY', 'TQZKVZQ', 'HGCJPXQ', 'MAODOCR', 'QTIPFME', 'ERXNSUN', 'BGYZPRZ', 'BDABVCY', 'IYFABDO', 'MRDZBFN', 'AXVFXOV', 'ZAIRSDG', 'MZUTRGY', 'OELGKKQ', '

Transform all columns to numerical dtype (as they are all numerical but pd detected them as 'object' due to nans) and fill missing values

In [5]:
# I see al values are numerical so let's "fix" the types to make it easier to work with
# true - false values will become 1 and 0
df_filtered = df_filtered.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# Object types will transform to numerical types
for col in df_filtered.columns:
    if df_filtered[col].dtype == 'object':
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

# Fill NaNs with the mean of each column
df_filled = df_filtered.fillna(df_filtered.mean())

<ipython-input-5-d5f526ba9e72>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered = df_filtered.applymap(lambda x: 1 if x is True else (0 if x is False else x))


Next step to reduce number of columns is to removed highly correleted columnes (I have set a threshold of 0.9) - the idea is that if we have 2 corr columns then one is enought the learn from, and most likly the other won't share new information to the model.

In [6]:
# I use a sample of the numeric data for correlation matrix due to memory issues on my computer
correlation_matrix = df_filled.sample(frac=0.5).corr()

# I set a high threshold for correlation to be coutions
threshold = 0.9

# Identify feature pairs with correlation higher than the threshold
high_correlation = correlation_matrix[(correlation_matrix > threshold) & (correlation_matrix != 1.0)]

# Find the column pairs
correlated_features = high_correlation.stack().reset_index()
correlated_features.columns = ['Feature1', 'Feature2', 'Correlation']

columns_to_drop = set()

# Iterate over the pairs and add one of the highly correlated columns to the drop list
for feature1, feature2, _ in correlated_features.values:
    columns_to_drop.add(feature2)

columns_to_drop = list(columns_to_drop)

# Drop the highly correlated columns from the DataFrame
df_reduced = df_filled.drop(columns=columns_to_drop)

print(f"Columns dropped due to high correlation: {columns_to_drop}")
print(f"Original shape: {df_filled.shape}")
print(f"Reduced shape: {df_reduced.shape}")

Columns dropped due to high correlation: ['FXACWUA', 'MQXCIBE', 'QALTYYP', 'UHMRZHI', 'LTDFQFZ', 'TXVCNZC', 'GOBJALH', 'MRBOALK', 'XLHIYUD', 'JGXPQNV', 'YRLOEIR', 'MOSECGI', 'NVBZJEU', 'OULTOYT', 'TKQUNLP', 'JYSKSPX', 'SKKXRWR', 'VFXSABJ', 'UZIGIGC', 'ZPBDTHO', 'HNXONUF', 'BBVLSAK', 'BGEQHWR', 'HXBFDMA', 'BZYFPMM', 'LCFCVCB', 'RLYWHCU', 'LGXZTHS', 'LNZGNNA', 'EYARVDG', 'FNCZPLL', 'SMHHBFH', 'MXNLYUW', 'HOVFLAR', 'TLOEEZC', 'JZJOQOG', 'SFTKCKG', 'VTLKBMI', 'BZFTTYE', 'JIQIQWD', 'BGPVFMN', 'FTMZZVG', 'DDZYPGM', 'VYTOISY', 'CFAIUJQ', 'QASZGHA', 'XIKNUBQ', 'DUOKWOI', 'OKIKPOJ', 'WBCMBKW', 'IKMWIOV', 'WWVGPWZ', 'PCYIKPD', 'KYJTYYF', 'CKBVBUJ', 'DVRDHUH', 'QODSZMV', 'DMTXDGF', 'OSFOBBT', 'BTXMHGA', 'VEUJYWN', 'BYEQVGG', 'QYDTWHL', 'GZGWPXA', 'SYJXQKZ', 'OUIMKJU', 'RHSKKTW', 'YLHQTEA', 'RRMMRAJ', 'BTKTNSN', 'WVRSHBW', 'VCPMKTP', 'GKACPXS', 'EEGYIGX', 'MRUWBZT', 'OXRCIYW', 'UWOLTWR', 'DIKRMOL', 'ZSGPKXD', 'SPKCFPP', 'UGXVMKE', 'QJKCJMC', 'WBBKQFH', 'VYQUVQJ', 'HEBFRRA', 'NOLZTQY', 'RXLZZHO', '

We droppped the number of columns from 649 to 239. Next step that I want to do is to select the K "best" features - meaning, features that have high score with the target variable. In my case, I chose to calc the score based on ANOVA F-value between label and feature.

In [7]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# First, I'll creat the binary label
df_reduced['TLJYWBE_results'] = df_reduced['TLJYWBE'].apply(lambda x: 1 if x < 1e-5 else 0)

# Define X and y
X = df_reduced.drop(['TLJYWBE', 'TLJYWBE_results'], axis=1).values
y = df_reduced['TLJYWBE_results'].values

# For classification, I use f_classif to calc the score
# I Select top 50 features (K = 50), but we can play with this
selector = SelectKBest(score_func=f_classif, k=50)
X_selected = selector.fit_transform(X, y)

# We can look at the features and scores:
scores = selector.scores_
feature_names = df_reduced.drop(['TLJYWBE', 'TLJYWBE_results'], axis=1).columns

# And create a DataFrame for easier interpretation
features_df = pd.DataFrame({
    'Feature': feature_names,
    'Score': scores
}).sort_values(by='Score', ascending=False)

features_df

<ipython-input-7-380fbe794f80>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reduced['TLJYWBE_results'] = df_reduced['TLJYWBE'].apply(lambda x: 1 if x < 1e-5 else 0)


,Feature,Score
192,HBQPQTD,117.959248
230,AHVIXII,20.188711
213,OMIIMXP,15.302103
28,VZOZKWX,12.935166
97,KTDHAGX,12.676015
...,...,...
44,WFPEMDD,0.000258
74,RSQGZAF,0.000051
42,UOWMANL,0.000025
205,TTQVFLL,0.000002


This part is not needed, but it's only for the purpse of saving the df that we are going to train on (I did sample sure to size + removed cols that we don't use for training), since I was request to save as CSV.

In [15]:
features_only = df_reduced.drop(['TLJYWBE', 'TLJYWBE_results'], axis=1)

# Apply the mask to select the columns
selected_columns = features_only.columns[selected_mask]

# Convert selected_columns to a list and add 'TLJYWBE' and 'TLJYWBE_results'
final_columns = list(selected_columns) + ['TLJYWBE', 'TLJYWBE_results']

# Create a new DataFrame with the selected columns + target columns
df_selected = df_reduced[final_columns]

# Save the DataFrame to a CSV file
df_selected.to_csv('df_for_training.csv', index=False)

### **Train & Eval Model**

I choose to train XGB Classifier (since we want to predict pass/fail). I didn't use normalization since XGB, which relies  on gradient boosting trees, is less sensitive to the scale of features. I would, however, check it in further step (but due to time limit I passed)

In [29]:
from sklearn.model_selection import train_test_split

X_selected = df_selected.drop(['TLJYWBE', 'TLJYWBE_results'], axis=1).values
y = df_reduced['TLJYWBE_results'].values

# Split the data into training and test sets
# I don't see any timestemp is the data so no need for time-splitting
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [30]:
# Handle class imbalance
# Calculate the scale_pos_weight
class_counts = pd.Series(y).value_counts()
scale_pos_weight = class_counts[0] / class_counts[1]
print("scale_pos_weight", scale_pos_weight)

scale_pos_weight 0.00010671844232384571


In [32]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, average_precision_score
import xgboost as xgb
import matplotlib.pyplot as plt

# Initialize XGBoost model with scale_pos_weight to handle class imbalance
model = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of class 1

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Precision, Recall, and F1 Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print(f'Precision-Recall AUC: {auc_pr:.4f}')

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.80      0.00         5
           1       1.00      0.97      0.98     58098

    accuracy                           0.97     58103
   macro avg       0.50      0.88      0.49     58103
weighted avg       1.00      0.97      0.98     58103

Confusion Matrix:
[[    4     1]
 [ 1905 56193]]
Precision: 1.0000
Recall: 0.9672
F1 Score: 0.9833
Precision-Recall AUC: 1.0000
